In [21]:
import rioxarray as rxr
import xarray as xr
# import rioxarray as rxr
import matplotlib.pyplot as plt

In [22]:
paths = ["data/20230708_ms_koc_index_blue.tif",
         "data/20230708_ms_koc_index_green.tif",
         "data/20230708_ms_koc_index_red.tif",
         "data/20230708_ms_koc_index_nir.tif",
         "data/20230708_ms_koc_index_red_edge.tif"]
bands = ['blue', 'green', 'red', 'nir', 'red_edge']

In [23]:
ds = xr.Dataset()
for path, band in zip(paths, bands):
    ds[band] = rxr.open_rasterio(path).squeeze()
ds = ds.rio.reproject("EPSG:4326", interpolation='nearest')

In [ ]:
ds.to_netcdf('data/multispectral.nc', engine='h5netcdf')

In [24]:
rgb_ds = ds[["red", "green", "blue"]]
rgb_da = rgb_ds.to_array(dim='band')
nodata_mask = rgb_da != -10000
nodata_mask = nodata_mask.all(dim='band')
rgb_da = rgb_da.where(nodata_mask)
#stretch from 0 to 1
rgb_da = rgb_da / rgb_da.max(dim=['x', 'y'])
nodata_mask = nodata_mask.expand_dims(dim='band').assign_coords(band=['alpha'])*255
rgb_da = (rgb_da * 255)
rgb_da = xr.concat([rgb_da, nodata_mask], dim='band')
rgb_da = rgb_da.astype('uint8')

/usr/local/lib/python3.11/dist-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [25]:
rgb_da.rio.to_raster('data/rgb.tif')